In [10]:
import rpyc
from rpyc.utils.server import Server, ThreadedServer
from rpyc import async_
from threading import Thread, Lock, RLock
import time
import copy


mt = RLock()

class Myservice(rpyc.Service):
    def __init__(self):
        self.ls_child = ls_child
        self.child_conn = []
        self.bgthreads = []
        self.func = None
        self.id_list = []
        self.async_objs = []
        self.l = []
        self.shadow = None
        self.gran = 5
        self.lc = 0
        self.id = 21
        self.counter = 1
        self.finishes = {}
        self.pkt_details = {}
        self.is_shadow = True
        
        for ch in self.ls_child:
            c = rpyc.connect(ch[0], ch[1])
            self.child_conn.append(c)
            self.bgthreads.append(rpyc.BgServingThread(c))
#             self.on_connect(c)
        if(self.is_shadow):
            self.sent_packets = []
        else:
            self.shadow = shadow
            self.shadow_conn = rpyc.connect(self.shadow[0], self.shadow[1])
            self.shadow_bg = rpyc.BgServingThread(self.shadow_conn)
    
            
    def on_finish(self):

        if(len(self.finishes) == (len(self.child_conn) + 1)):
            print("enter")
            print(self.lc)
            while(len(self.l) > self.lc):
                send = self.gran
                if (len(self.l) - self.lc < self.gran):
                    send = len(self.l) - self.lc
                self.pkt_details[str(self.counter)] = (self.lc, send)
                self.func(self.counter, self.l[self.lc:self.lc+send], self.id, self.ack_func)
                self.counter += 1
                print("on_finish waala {}".format(self.counter))
                self.lc += send
    
    def exposed_packet_ack(self, l):
        self.sent_packets.extend(l)
        print(l)
        
    def exposed_print_pkt_list(self):
        print(self.sent_packets)
    
    def ack_func(self, ID):
        print("Ack recieved for packet id {}".format(ID))
        vals = self.pkt_details[str(ID)]
        self.shadow_conn.root.packet_ack(self.id_list[vals[0]:vals[0]+vals[1]])
    
    
    def cback(self, pkt_id, value, ID, ack_func, fin = False):
        global mt
#         print("Before acquiring by " + str(ID))
        #print(mt.locked())
        mt.acquire()
        try:
            if(fin == True):
                print("fin called by {}".format(ID))
                self.finishes[str(ID)] = True
                self.on_finish()
            else:                
# #                 print("in cback " + str(ID))
                idl = []
                
                for _ in range(len(value)):
                    idl.append(ID)

                self.id_list.extend(idl)
        #         print(value)
                #print(mt.locked())
                
#                 print(len(value))
                self.l.extend(value)
#                 print(pkt_id, ID, self.id)
                if (ID != self.id):
                    ack_func(pkt_id)
#                 print("lala")
#                 print("Value written")
#                 print (len(self.l))

                while(len(self.l) - self.lc >= self.gran):
    #                 print('andar {}'.format(ID))
                    self.pkt_details[str(self.counter)] = (self.lc, self.gran)
                    self.func(self.counter, self.l[self.lc:self.lc+self.gran], self.id, self.ack_func)
#                     print(str(self.counter), self.pkt_details[str(self.counter)])
                    self.counter += 1
                    print("cback waala {}".format(self.counter))
                    self.lc += self.gran
                    
#                 print(len(self.l) - self.lc)
#             print('in cback print {}'.format(ID))
#             print(self.l)
#             print("pople")
            
        except Exception as e:
            print(e)
        finally:
            mt.release()
#             print("mutex released")
        
        return
    
    def on_connect(self, conn):
        print("Connection established with "+ str(conn))
        
    def on_disconnect(self,conn):
        print("Have a nice day... goodbye")
        
    def exposed_printvals(self):
        print(self.l)
        print(self.id_list)
    
    def local_search(self):
        fn = self.cback
        for j in range(10):
            l = []
            for i in range(200 + 10*j,200 + 10*j + 10):
                l.append(i)


            time.sleep(1)
            print("done " + str(j))
            fn(j, l, self.id, self.ack_func)
            print("sent " + str(j))
        
        fn(None, None, self.id, None, True)   
            
    def exposed_do_a_search(self, func):
        global mt
        
        self.func = func
#         for child in self.child_conn:
#             ID = child.root.get_id()
#             self.mapper[str(ID)] = child
            
        for child in self.child_conn:
            asearch = async_(child.root.do_a_search)
            aobj = asearch(self.cback)
            self.async_objs.append(aobj)

        
        self.local_search()
#         if(self.is_shadow == False):
#             self.shadow_conn.root.print_pkt_list()
        
#         while True:
#             time.sleep(2)
#             mt.acquire()
#             try:
#                 flag = 1
#                 for _, value in self.finishes.items():
#                     if(value == False):
#                         flag = 0
#                         break
#                 if(flag == 1):
# #                     while(len(self.l) > self.lc):
# #                         send = self.gran
# #                         if (len(self.l) - self.lc < self.gran):
#                         send = len(self.l) - self.lc
#                         self.func(self.l[self.lc:self.lc+send], self.id_list[self.lc:self.lc+send])
#                         self.counter += 1
#                         print(self.counter)
#                         self.lc += send
                        
#             finally:
#                 mt.release()
        
#         while True:
#             time.sleep(1)
# #             print("lolol")
#             print("Before acquiring - while")
#             #print(mt.locked())
#             mt.acquire()
#             try:
#                 print("in while")
#                 #print(mt.locked())
#                 if(len(self.l)>=2 and times <=10):
#                     print("1")
#                     times += 1
#     #                 id1 = copy.deepcopy(self.id_list)
#     #                 print(id1)
#                     print("Sending pac")
# #                     print("in while print " +  str(times) + " " + str(self.l) )
#     #                 l1 = copy.deepcopy(self.l)
#                     print("blah")
#                     func(self.l[lc:lc+2], self.id_list[lc:lc+2])
#                     lc += 2
#             except Exception as e:
#                 print(e)
#             finally:
#                 mt.release()
#             if(times > 10):
#                 break
#             print("while mutex released")    

In [11]:
ls_child = []
ls_child.append(("localhost", 18850, 11))
ls_child.append(("localhost", 18851, 12))

shadow = ["localhost", 17752]

In [ ]:
t = ThreadedServer(Myservice, port = 17752)
t.start()

Connection established with <rpyc.core.protocol.Connection 'conn21' object at 0x7f15b5588748>
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[11, 11, 11, 11, 12]
[12, 12, 12, 12, 12]
[12, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 11, 11, 11, 11]
[12, 12, 12, 12, 12]
[12, 12, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 11, 11, 11]
[11, 12, 12, 12, 12]
[12, 12, 12, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 11, 11]
[11, 11, 12, 12, 12]
[12, 12, 12, 12, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 11]
[11, 11, 11, 12, 12]
[12, 12, 12, 12, 12]
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[11, 11, 11, 11, 12]
[12, 12, 12, 12, 12]
[12, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 11, 11, 11, 11]
[12, 12, 12, 12, 12]
[12, 12, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 11, 11, 11]
[11, 12, 12, 12, 12]
[12, 12, 12, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 11, 11]
[11, 11, 12, 12, 12]
[12, 12, 12, 12, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 11]
[11, 11, 11, 12, 12]
[12, 12, 12, 12, 12]


Exception in thread Thread-24:
Traceback (most recent call last):
  File "/home/vishal/.pyenv/versions/3.5.2/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/vishal/.pyenv/versions/3.5.2/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/vishal/.pyenv/versions/python-3.5.2/lib/python3.5/site-packages/rpyc/utils/helpers.py", line 210, in _bg_server
    self._conn.serve(self.SERVE_INTERVAL)
  File "/home/vishal/.pyenv/versions/python-3.5.2/lib/python3.5/site-packages/rpyc/core/protocol.py", line 387, in serve
    data = self._channel.poll(timeout) and self._channel.recv()
  File "/home/vishal/.pyenv/versions/python-3.5.2/lib/python3.5/site-packages/rpyc/core/channel.py", line 50, in recv
    header = self.stream.read(self.FRAME_HEADER.size)
  File "/home/vishal/.pyenv/versions/python-3.5.2/lib/python3.5/site-packages/rpyc/core/stream.py", line 230, in read
    raise EOFError("connection closed b

Connection established with <rpyc.core.protocol.Connection 'conn24' object at 0x7f15b55e5c50>
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[11, 11, 11, 11, 12]
[12, 12, 12, 12, 12]
[12, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 11, 11, 11, 11]
[12, 12, 12, 12, 12]
[12, 12, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 11, 11, 11]
[11, 12, 12, 12, 12]
[12, 12, 12, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 11, 11]
[11, 11, 12, 12, 12]
[12, 12, 12, 12, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 11]
[11, 11, 11, 12, 12]
[12, 12, 12, 12, 12]
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[11, 11, 11, 11, 12]
[12, 12, 12, 12, 12]
[12, 21, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 11, 11, 11, 11]
[12, 12, 12, 12, 12]
[12, 12, 21, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 11, 11, 11]
[11, 12, 12, 12, 12]
[12, 12, 12, 21, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 11, 11]
[11, 11, 12, 12, 12]
[12, 12, 12, 12, 21]
[21, 21, 21, 21, 21]
[21, 21, 21, 21, 11]
[11, 11, 11]
